# INTERNET_CSV

## Overview
The `INTERNET_CSV` function allows you to import data from any public CSV file on the internet directly into Excel. By providing a URL to a CSV file, the function fetches the file, parses its contents, and returns the data as a 2D list (range) suitable for use in Excel formulas. This is meant primarily as an example you can build off of for functions requiring access to data on the internet.

The function automatically prepends the URL with `https://cors.boardflare.com/` before fetching the file.  This proxies the request and appends the [CORS](https://developer.mozilla.org/docs/Web/HTTP/CORS) headers required by the browser.

## Usage
To use the `INTERNET_CSV` function in Excel, enter it as a formula in a cell, specifying the direct URL to the CSV file you want to import:

```excel
=INTERNET_CSV(url)
```

## Arguments
| Argument | Type   | Required | Description                                      | Example |
|----------|--------|----------|--------------------------------------------------|---------|
| url      | string | Yes      | The direct URL to the CSV file you want to import.| "https://files.zillowstatic.com/research/public_csvs/market_temp_index/Metro_market_temp_index_uc_sfrcondo_month.csv?t=1746621339" |

## Returns
| Returns | Type         | Description                                                      | Example |
|---------|--------------|------------------------------------------------------------------|---------|
| Data    | List[List[str]] | 2D list containing the parsed CSV data. Each row is a list of strings. | [["RegionID", "SizeRank", ...], ["102001", "0", ...], ...] |
| Error   | string      | Error message if an exception occurs                              | "Error: Invalid input" |

## Limitations
- Only supports public CSV files accessible via direct URL.
- Assumes the CSV file is properly formatted and accessible.
- Designed for educational and automation purposes; not for sensitive data.

## Benefits
- Enables seamless import of live, external datasets into Excel.
- Reduces manual data download and copy-paste steps.
- Flexible for a variety of public data sources.

## Examples

### Importing Zillow Market Heat Index Data
**Sample Call:**
```excel
=INTERNET_CSV("https://files.zillowstatic.com/research/public_csvs/market_temp_index/Metro_market_temp_index_uc_sfrcondo_month.csv?t=1746621339")
```
**Sample Output:**
| RegionID | SizeRank | RegionName      | RegionType | StateName | 2018-01-31 | 2018-02-28 |
|----------|----------|-----------------|------------|-----------|------------|------------|
| 102001   | 0        | United States   | country    |           | 50         | 50         |
| 394913   | 1        | New York, NY    | msa        | NY        | 52         | 52         |
| 753899   | 2        | Los Angeles, CA | msa        | CA        | 69         | 67         |
| 394463   | 3        | Chicago, IL     | msa        | IL        | 48         | 49         |

### Importing Zillow New Construction Sales Data
**Sample Call:**
```excel
=INTERNET_CSV("https://files.zillowstatic.com/research/public_csvs/new_con_sales_count_raw/Metro_new_con_sales_count_raw_uc_sfrcondo_month.csv?t=1746621339")
```
**Sample Output:**
| RegionID | SizeRank | RegionName      | RegionType | StateName | 2018-01-31 | 2018-02-28 |
|----------|----------|-----------------|------------|-----------|------------|------------|
| 102001   | 0        | United States   | country    |           | 34508      | 33782      |
| 394913   | 1        | New York, NY    | msa        | NY        | 532        | 447        |
| 753899   | 2        | Los Angeles, CA | msa        | CA        | 333        | 398        |
| 394463   | 3        | Chicago, IL     | msa        | IL        | 288        | 255        |

In [ ]:
import csv
import io
import requests

def internet_csv(url):
    """
    Fetches a CSV file from the internet (with CORS proxy) and returns its contents as a 2D list.
    Args:
        url (str): The direct URL to the CSV file.
    Returns:
        list[list[str]]: 2D list representing the CSV contents, where each sublist is a row.
    Raises:
        ValueError: If the URL is invalid or the content cannot be parsed as CSV.
    """
    if not isinstance(url, str) or not url.strip():
        raise ValueError("A non-empty URL string must be provided.")
    cors_url = f"https://cors.boardflare.com/{url}"
    try:
        response = requests.get(cors_url)
        response.raise_for_status()
        content = response.text
        reader = csv.reader(io.StringIO(content))
        data = [row for row in reader]
        if not data:
            raise ValueError("CSV file is empty or could not be parsed.")
        return data
    except Exception as e:
        raise ValueError(f"Failed to fetch or parse CSV: {e}")

In [ ]:
# Unit Tests
import ipytest
ipytest.autoconfig()

def test_demo_zillow_market_heat_index():
    url = "https://files.zillowstatic.com/research/public_csvs/market_temp_index/Metro_market_temp_index_uc_sfrcondo_month.csv?t=1746621339"
    result = internet_csv(url)
    assert isinstance(result, list)
    assert all(isinstance(row, list) for row in result)
    assert len(result) >= 4
    assert result[0][:4] == ["RegionID", "SizeRank", "RegionName", "RegionType"]

def test_demo_zillow_new_construction_sales():
    url = "https://files.zillowstatic.com/research/public_csvs/new_con_sales_count_raw/Metro_new_con_sales_count_raw_uc_sfrcondo_month.csv?t=1746621339"
    result = internet_csv(url)
    assert isinstance(result, list)
    assert all(isinstance(row, list) for row in result)
    assert len(result) >= 4
    assert result[0][:4] == ["RegionID", "SizeRank", "RegionName", "RegionType"]

def test_empty_url():
    try:
        internet_csv("")
        assert False, "Should raise ValueError for empty URL"
    except Exception:
        pass

def test_invalid_url():
    try:
        internet_csv("not_a_url")
        assert False, "Should raise ValueError for invalid URL"
    except Exception:
        pass

ipytest.run()

In [ ]:
# Interactive Demo
import gradio as gr

examples = [
    ["https://files.zillowstatic.com/research/public_csvs/market_temp_index/Metro_market_temp_index_uc_sfrcondo_month.csv?t=1746621339"],
    ["https://files.zillowstatic.com/research/public_csvs/new_con_sales_count_raw/Metro_new_con_sales_count_raw_uc_sfrcondo_month.csv?t=1746621339"]
]

demo = gr.Interface(
    fn=internet_csv,
    inputs=gr.Textbox(label="CSV URL", value="https://files.zillowstatic.com/research/public_csvs/market_temp_index/Metro_market_temp_index_uc_sfrcondo_month.csv?t=1746621339"),
    outputs=gr.Dataframe(label="CSV Data"),
    examples=examples,
    description="Import data from any public CSV file on the internet into Excel-compatible format.",
    flagging_mode="never",
)
demo.launch()